In [1]:
using HTTP, JSON, PrettyTables, JLD, DotEnv,  Random, Dates
institutions = load("institutions.jld")["institutions"]
academic_list = load("academic_list.jld")["academic_list"]
adjacency = load("adjacency_2023.jld")["out"];

In [2]:
C = zeros(Int32,12,length(institutions))
T = zeros(Int32, length(academic_list), 5);

In [3]:
# fill in the hard coded sinks
est_alloc = load("est_alloc.jld")["est_alloc"];
for i in length(academic_list)+1:length(institutions)
    C[est_alloc[i], i] = 1
end

In [4]:
# sinks have been hard coded - start by assigning all academic institutions to community 1
for i in 1:length(academic_list)
    T[i,1] = 1
    C[1,i] = 1
end
#now all academic institutions are in the same tier

In [5]:
sum(C)

991

In [6]:
sum(T)

412

In [7]:
c = sum(C, dims = 2)

12×1 Matrix{Int64}:
 412
   0
   0
   0
   0
  66
  69
 188
  50
   1
  13
 192

In [8]:
t = sum(T, dims=1)

1×5 Matrix{Int64}:
 412  0  0  0  0

In [9]:
C*adjacency*T

12×5 Matrix{Int32}:
 650  0  0  0  0
   0  0  0  0  0
   0  0  0  0  0
   0  0  0  0  0
   0  0  0  0  0
 153  0  0  0  0
 146  0  0  0  0
  81  0  0  0  0
  12  0  0  0  0
 266  0  0  0  0
  52  0  0  0  0
 236  0  0  0  0

In [10]:
function likelihood(adjacency, c, t)
    # the number of institutions in each tier
    # the number of academic institions in each tier
    l = 0.0
    for i in 1:12,j in 1:5
        l +=  -adjacency[i,j]*(log(max(adjacency[i,j]/(max(c[i]*t[j],1)),.0001)) - 1)
    end
    return l
end
    

likelihood (generic function with 1 method)

In [11]:
likelihood(C*adjacency*T, c, t)

9150.45365384631

In [12]:
sum(adjacency)

1596

In [13]:
function best_alloc(C,T,adjacency, iterations = 10) 
    c = sum(C, dims = 2)
    t = sum(T, dims=1)
    start = likelihood(C*adjacency*T, c, t)
    sq = [C, T, start]
    println("Starting value: ", start)
    n = 0
    while n < iterations
        shuffle = []
        to_tier = []
        new_C = zeros(Int32,size(C,1),size(C,2))
        new_T = zeros(Int32, size(T,1),size(T,2))
        for j in 1:1
            push!(shuffle, rand(1:size(T,1)))
            push!(to_tier, rand(1:size(T,2)))
        end
    
        for i in 1:size(C,2), j in 1:size(C,1)
            index = findfirst(x -> x == i, shuffle)
            if !isnothing(index)
              #  println(shuffle)
              #  println(to_tier)
              #  println(index)
              #  println("index ",shuffle[index], " to tier ", to_tier[index], " i ", i , " j ", j)
                
                
                if j == to_tier[index] 
               #     println("resetting index ",j," ",i," to one")
                #    return 1
                    new_C[j, i] = 1
                    new_T[i, j] = 1
                else
                    new_C[j, i] = 0
                 #   println("Reset ",C[j,i]," at ", j," ",i, " to zero")
                    if j < size(T,2) + 1
                        new_T[i, j] = 0
                    end
                end
            else
                new_C[j,i] = sq[1][j,i]
                if i < size(T,1) + 1
                    if j < 6
                        new_T[i, j] = sq[2][i,j]
                    end
                end
            end
        end
        c = sum(new_C, dims = 2)
        t = sum(new_T, dims = 1)
        new_l = likelihood(new_C*adjacency*new_T,c,t)
        
        if new_l < sq[3]
            println("Iteration ", n, ": " , new_l)
            sq[3] = new_l
            sq[1] = new_C
            sq[2] = new_T
        end
        n += 1
    end
                
    return sq
end
        

best_alloc (generic function with 2 methods)

In [14]:
sq = best_alloc(C,T,adjacency, 1000);



Starting value: 9150.45365384631
Iteration 0: 9147.529092800485
Iteration 1: 9143.791608252148
Iteration 2: 9141.230591561234
Iteration 4: 9136.32592608019
Iteration 5: 9133.756029685172
Iteration 7: 9133.652302110717
Iteration 8: 9130.555983245411
Iteration 10: 9130.0419998404
Iteration 11: 9126.920163930212
Iteration 13: 9122.892043133255
Iteration 16: 9121.006979669966
Iteration 17: 9117.956520037127
Iteration 23: 9114.459891983268
Iteration 25: 9110.959008518668
Iteration 27: 9109.121532923322
Iteration 28: 9107.838069634668
Iteration 30: 9105.725178781624
Iteration 32: 9104.3602959039
Iteration 34: 9101.027455801543
Iteration 35: 9099.102599741384
Iteration 37: 9098.61337033338
Iteration 39: 9094.759223556946
Iteration 41: 9093.691511059458
Iteration 42: 9089.813890873265
Iteration 45: 9088.089476876607
Iteration 48: 9085.761642045993
Iteration 49: 9082.390007553902
Iteration 51: 9077.052266564702
Iteration 53: 9074.363185452536
Iteration 54: 9070.325351343412
Iteration 55: 9069.4

In [18]:
sq[1]*adjacency*sq[2]

12×5 Matrix{Int32}:
 488   7  14  11  17
  35  17   7   1   4
   8   0  13   1   0
   6   0   3   6   0
   7   0   0   0   5
 135   2   8   7   1
 136   1   5   1   3
  45  17  14   1   4
   7   3   0   2   0
 160  22  13  30  41
  48   0   1   3   0
 184  15   5  20  12

In [15]:
for i in 1:size(adjacency,2)
    if sq[2][i,2] == 1
        println(institutions[i])
    end
end

Australian National University
Azerbaijan State University of Economics
Brunel University London
CERGE-EI (Center for Economic Research and Graduate Education – Economics Institute)
Cardiff University
Central University of Finance and Economics
Collegio Carlo Alberto
Colorado State University
Dalhousie University
Devi Ahilya Vishwavidyalaya, Indore
Durham University
George Mason University
Govind Ballabh Pant University of Agriculture and Technology
Hamburg University
Instituto Tecnologico Autónomo de Mexico (ITAM)
Kyoto University
Makerere University
Nankai University
Nanyang Technological University
Nigerian Defence Academy
Seoul National University
Shiv Nadar University
Tata Institute of Social Sciences
Trinity College Dublin, University of Dublin
Universidad del País Vasco / Euskal Herriko Unibertsitatea
University College Dublin
University at Buffalo
University of Bern
University of Delhi
University of Edinburgh
University of Exeter
University of Groningen
University of Huddersfie

In [16]:
sum(sq[2], dims = 1)

1×5 Matrix{Int64}:
 191  48  49  59  65